In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from relational_embedder.data_prep import data_prep_utils as dpu
import numpy as np
from relational_embedder import api
from relational_embedder.api import Fabric
from relational_embedder.api import SIMF
import word2vec
import pickle
import itertools
path_to_row_we_model = '/data/raulcf/relemb/mitdwh/bench_mitdwh/we_db2vec_75_ns50_fns20_i40.bin'
row_we_model = word2vec.load(path_to_row_we_model)
path_to_row_relemb = '/data/raulcf/relemb/mitdwh/bench_mitdwh/relemb_db2vec_75_ns50_fn20/row.pkl'
word_hubness_path = "/data/raulcf/relemb/mitdwh/bench_mitdwh/relemb_db2vec_75_ns50_fn20/hubness.pkl"
with open(word_hubness_path, "rb") as f:
    word_hubness = pickle.load(f)
with open(path_to_row_relemb, "rb") as f:
    row_relemb = pickle.load(f)
fabric = Fabric(row_we_model, None, row_relemb, None, None, word_hubness)

In [3]:
# vis imports
#import matplotlib
#matplotlib.use('Agg')
import hypertools as hyp

/home/raulcf/f/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
list_of_arrays = fabric.M_R.vectors[:100]

In [5]:
hyp.plot(list_of_arrays, '.')

TclError: couldn't connect to display "istc4:0.0"